In [1]:
from equilibrator_api import ComponentContribution, Reaction
import numpy as np
import json
import gzip
import pandas as pd

with gzip.open("kegg_enzymes.json.gz", "r") as f:
        ECs = {e['EC']:e['reaction_ids'] for e in json.load(f)}

with gzip.open("kegg_reactions.json.gz", "r") as f:
        RXNs = {r['RID']:r['reaction'] for r in json.load(f)}

cc = ComponentContribution()
list_of_ECs = [list(ECs.keys())[i] for i in [5,6,10,12,23]]
model_ECs = pd.read_csv('Reaction.csv', usecols=['EC'])


In [2]:
db = pd.DataFrame(columns=['EC','KEGG Rid','R string','Keq'])
for ec_string in model_ECs['EC'][20:25]:
    print(ec_string)
    try:
        for r in ECs[ec_string]:
            print(r)
            try:
                rxn = Reaction({cc.get_compound("kegg:"+species[1]):species[0] for species in RXNs[r]})
                if not rxn.is_balanced():
                    rxn = cc.balance_by_oxidation(rxn)
                    print('balancing!')
                r_string = rxn.__str__()
                Keq = np.exp((-cc.standard_dg_prime(rxn)/cc.RT).value).magnitude 
                print(cc.standard_dg_prime(rxn))

            except:
                r_string = ''
                Keq = np.nan

            db = pd.concat([pd.DataFrame([[ec_string,r,r_string,Keq]], columns=db.columns), db], ignore_index=True)

    except:
        r = ''
        r_string = ''
        Keq = np.nan
        
        db = pd.concat([pd.DataFrame([[ec_string,r,r_string,Keq]], columns=db.columns), db], ignore_index=True)

print(db)
# rxn = cc.search_reaction(reaction_string)
# sp = [c.inchi_key for c in list(rxn.sparse_with_phases.keys())]
# print(cc.standard_dg_prime(rxn))
# Keq = np.exp((-cc.standard_dg_prime(rxn)/cc.RT).value)
# print(Keq)

6.1.1.15
R03661
balancing!


c:\Users\Diego Alba\.conda\envs\ODBM2\lib\site-packages\equilibrator_cache\reaction.py:616: UserWarning: Cannot calculate Legendre transform for Compound(id=74548, inchi_key=None): TRNA(PRO) has not yet been analyzed by ChemAxon.
  warnings.warn(
c:\Users\Diego Alba\.conda\envs\ODBM2\lib\site-packages\equilibrator_cache\reaction.py:616: UserWarning: Cannot calculate Legendre transform for Compound(id=234, inchi_key=None): Pro-tRNA(Pro) has not yet been analyzed by ChemAxon.
  warnings.warn(


(-0.0 +/- 1.0)e+05 kilojoule / mole
1.11.1.26
6.1.1.5
R03656
balancing!


c:\Users\Diego Alba\.conda\envs\ODBM2\lib\site-packages\equilibrator_cache\reaction.py:616: UserWarning: Cannot calculate Legendre transform for Compound(id=74587, inchi_key=None): tRNA(Ile) has not yet been analyzed by ChemAxon.
  warnings.warn(
c:\Users\Diego Alba\.conda\envs\ODBM2\lib\site-packages\equilibrator_cache\reaction.py:616: UserWarning: Cannot calculate Legendre transform for Compound(id=74418, inchi_key=None): Ile-tRNA(Ile) has not yet been analyzed by ChemAxon.
  warnings.warn(


(-0.0 +/- 1.0)e+05 kilojoule / mole
2.7.9.2
R00199
(-4.9 +/- 0.5) kilojoule / mole
2.3.1.61
R02570
balancing!


c:\Users\Diego Alba\.conda\envs\ODBM2\lib\site-packages\equilibrator_cache\reaction.py:616: UserWarning: Cannot calculate Legendre transform for Compound(id=75258, inchi_key=None): CHEBI:80219 has not yet been analyzed by ChemAxon.
  warnings.warn(
c:\Users\Diego Alba\.conda\envs\ODBM2\lib\site-packages\equilibrator_cache\reaction.py:616: UserWarning: Cannot calculate Legendre transform for Compound(id=4798, inchi_key=None): CHEBI:80403 has not yet been analyzed by ChemAxon.
  warnings.warn(


(0.0 +/- 1.0)e+05 kilojoule / mole
R01933
(-33 +/- 4) kilojoule / mole
R02571
R08549
(-28 +/- 4) kilojoule / mole
          EC KEGG Rid                                           R string  \
0   2.3.1.61   R08549      NAD + CoA + 2OG <=> H+ + DPNH + CO2 + suc-coa   
1   2.3.1.61   R02571                                                      
2   2.3.1.61   R01933  NAD + CoA + oxoadipate <=> H+ + DPNH + CO2 + g...   
3   2.3.1.61   R02570  H+ + 3.5 O2 + suc-coa + CHEBI:80219 <=> 2 HO- ...   
4    2.7.9.2   R00199                 HOH + atp + PYR <=> Pi + AMP + PEP   
5    6.1.1.5   R03656  atp + 7.5 O2 + Ile + tRNA(Ile) <=> H+ + 4 H4O(...   
6  1.11.1.26                                                               
7   6.1.1.15   R03661  atp + 5.5 O2 + PRO + TRNA(PRO) <=> H+ + 2 H4O(...   

            Keq  
0  8.822845e+04  
1           NaN  
2  5.530758e+05  
3  5.463841e-10  
4  7.370111e+00  
5  4.543658e+08  
6           NaN  
7  4.616016e+08  


In [11]:
  
db.to_csv('parsed_w_strings.csv')

In [13]:
cc.ccache.get_compound_by_internal_id(41600)

Compound(id=41600, inchi_key=OSBNUHNVJFXWOG-WBHMSIDWSA-L)

In [14]:
cc.ccache.get_compound_by_inchi('AHKZTVQIVOEVFO-UHFFFAOYSA-N')

In [17]:
RXNs['R00345']

[[1, 'C00001'], [-1, 'C00009'], [1, 'C00011'], [-1, 'C00036'], [1, 'C00074']]

In [29]:
cc.get_compound("kegg:C00099").identifiers

def find_short(ids):
    shortest = 'dummy'*99
    for i in ids:
        if i.registry.name == "Synonyms":
            if len(i.accession) < len(shortest):
                shortest = i.accession
    return shortest

find_short(cc.get_compound("kegg:C00099").identifiers)

'bAla'

In [3]:
cc.get_compound("kegg:C00099").identifiers

[CompoundIdentifier(registry=Registry(namespace=bigg.metabolite), accession=ala_B),
 CompoundIdentifier(registry=Registry(namespace=bigg.metabolite), accession=M_ala_B),
 CompoundIdentifier(registry=Registry(namespace=chebi), accession=CHEBI:16958),
 CompoundIdentifier(registry=Registry(namespace=chebi), accession=CHEBI:10343),
 CompoundIdentifier(registry=Registry(namespace=chebi), accession=CHEBI:12389),
 CompoundIdentifier(registry=Registry(namespace=chebi), accession=CHEBI:22821),
 CompoundIdentifier(registry=Registry(namespace=chebi), accession=CHEBI:41050),
 CompoundIdentifier(registry=Registry(namespace=chebi), accession=CHEBI:57966),
 CompoundIdentifier(registry=Registry(namespace=chebi), accession=CHEBI:63070),
 CompoundIdentifier(registry=Registry(namespace=hmdb), accession=HMDB00056),
 CompoundIdentifier(registry=Registry(namespace=kegg), accession=C00099),
 CompoundIdentifier(registry=Registry(namespace=kegg), accession=D07561),
 CompoundIdentifier(registry=Registry(namespa

495

In [3]:
db

,EC,KEGG Rid,R string,Keq
0,2.3.1.61,R08549,DPN + CoA + akg <=> H + NADH + co2 + SUC-COA,8.822845e+04
1,2.3.1.61,R02571,,NaN
2,2.3.1.61,R01933,DPN + CoA + 2oxoadp <=> H + NADH + CO2 + glutcoa,5.530758e+05
3,2.3.1.61,R02570,H + 3.5 O2 + SUC-COA + Enzyme N6-(dihydrolipoy...,5.463841e-10
4,2.7.9.2,R00199,oh1 + ATP + PYR <=> Pi + amp + PEP,7.370111e+00
5,6.1.1.5,R03656,ATP + 7.5 O2 + I + ILE-tRNAs <=> H + 4 oxide +...,4.543658e+08
6,1.11.1.26,,,NaN
7,6.1.1.15,R03661,ATP + 5.5 O2 + P + a tRNApro <=> H + 2 oxide +...,4.616016e+08
